In [ ]:
import sys
sys.path.append('../../../')

import solvers.mechanism as mechanism
from solvers.mechanism import (
    State, Current, Parameter,
    state_derivative, state_steadystate, state_timeconst,
    mV, ms, S, cm2, nodim,
    v,
)
import numpy as np
from numpy import exp, log
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
class mech:
    """
    NaP channel mechanism
    """
    gmax  = 0.001 * S/cm2
    ena = 50.0 * mV

    Q10 = 3.0 * nodim

    # Activation & Deactivation (m-gate)
    theta_m0 = -50.0 * mV
    k_m = 5.7 * mV
    phi_m = -41.6 * mV
    sigma_m = 14.4 * mV
    alp0 = 2.130 * 1/ms
    bet0 = 2.460 * 1/ms

    # Slow Inactivation (s-gate)
    h0 = 0.154 * nodim
    theta_h = -57.0 * mV
    k_h = -4.0 * mV
    tau_h0 = 10.0 * ms
    tau_h1 = 17.0 * ms
    phi_h = -34.0 * mV
    sigma_h0 = -26.0 * mV
    sigma_h1 = -31.9 * mV

    # Slow Inactivation (s-gate)
    theta_s = -10.0 * mV
    k_s = -4.9 * mV
    Aa_s = -2.88e-6 * 1/ms/mV
    Ba_s = -4.9e-5 * 1/ms
    Ka_s = 4.63 * mV
    Ab_s = 6.94e-6 * 1/ms/mV
    Bb_s = 4.47e-4 * 1/ms
    Kb_s = -2.63 * mV

# m gate

In [ ]:
T_Q10 = mech.Q10 ** ((35-22)/10)

theta_m = mech.theta_m0 + (mech.k_m * log((1.0 / pow(0.5, 1.0/3.0)) - 1.0))

def minf(v):
    return 1.0 / (1.0 + exp((theta_m - v)/mech.k_m))

def alpha_m(v):
    return mech.alp0 * exp((v - mech.phi_m) / mech.sigma_m) * T_Q10

def beta_m(v):
    return mech.bet0 * exp((mech.phi_m - v) / mech.sigma_m) * T_Q10

In [ ]:
v = np.linspace(-100,100,400) * mV

plt.figure()
plt.plot(v, minf(v), 'r')
plt.plot(v, alpha_m(v)/(alpha_m(v)+beta_m(v)), 'g')

plt.figure()
plt.plot(v, 1/(alpha_m(v)+beta_m(v)))
plt.ylim((0.0, 0.1))
plt.xlabel('V (mV)')
plt.ylabel(r'$\tau_m$ (ms)')

In [ ]:
taum = 1/(alpha_m(v)+beta_m(v))

In [ ]:
min(taum)

In [ ]:
max(1./taum)

In [ ]:
theta_m

In [ ]:
T_Q10

# m gate (Kitano mod file)

In [ ]:
theta_m = -57.7 * (mV)
k_m = 5.7 * (mV)
tau_m0 = 0.03 * (ms)
tau_m1 = 0.146 * (ms)
phi_m = -42.6 * (mV)
sigma_m0 = 14.4 * (mV)
sigma_m1 = -14.4 * (mV)

h0 = 0.154
theta_h = -57.0 * (mV)
k_h = -4.0 * (mV)
tau_h0 = 10.0 * (ms)
tau_h1 = 17.0 * (ms)
phi_h = -34.0 * (mV)
sigma_h0 = 26.0 * (mV)
sigma_h1 = -31.9 * (mV)

theta_s = -10.0 * (mV)
k_s = -4.9 * (mV)
Aa_s = -2.88e-6 * (1/ms/mV)
Ba_s = -4.9e-5 * (1/ms)
Ka_s = 4.63 * (mV)
Ab_s = 6.94e-6 * (1/ms/mV)
Bb_s = 4.47e-4 * (1/ms)
Kb_s = -2.63 * (mV)

m_inf = 1.0 / (1.0 + exp((theta_m - v)/k_m))
tau_m = tau_m0 + (tau_m1 - tau_m0)/(exp((phi_m - v)/sigma_m0) + exp((phi_m - v)/sigma_m1))

In [ ]:
plt.figure()
plt.plot(v, m_inf, 'r')
plt.plot(v, minf(v), 'g')

plt.figure()
plt.plot(v, tau_m, 'b', label='Kitano')
plt.plot(v, 1/(alpha_m(v)+beta_m(v)), 'g', label='Edgerton')
plt.ylim(0.0, 0.1)
plt.xlabel('V (mV)')
plt.ylabel(r'$\tau_m$ (ms)')
plt.legend()

# m gate (Moyer mod file)

https://senselab.med.yale.edu/modeldb/showmodel.cshtml?model=150284

In [ ]:
mvhalf = -52.6 *(mV) # Magistretti 1999, Fig 4
mslope = -4.6 *(mV) # Magistretti 1999, Fig 4
minf = 1 / (1 + exp( (v - mvhalf) / mslope))
def mtau(v):
    v = v.m
    tau = np.empty_like(v)
    # Traub 2003, Table A2
    vlo = v<-40
    tau[vlo] = 0.025 + 0.14 * exp( (v[vlo] + 40 ) / 10)
    vhi = v>=-40
    tau[vhi] = 0.02 + 0.145 * exp( (-v[vhi] - 40) / 10)
    return tau

In [ ]:
plt.figure()
plt.plot(v, tau_m, 'b', label='Kitano')
plt.plot(v, 1/(alpha_m(v)+beta_m(v)), 'g', label='Edgerton')
plt.plot(v, mtau(v), 'r', label='Moyer')
plt.ylim(0.0, 0.1)
plt.xlabel('V (mV)')
plt.ylabel(r'$\tau_m$ (ms)')
plt.legend()

# m gate (Hay mod file)

https://senselab.med.yale.edu/modeldb/showmodel.cshtml?model=149100&file=%2fhayetal2013%2fnap_et2.mod#tabs-2

In [ ]:
minf_hay = 1.0/(1+exp((v.m- -52.6)/-4.6))
def mtau_hay(v, celsius=34):
    v = np.array(v.m)
    v[v==-38.0] += 0.0001
    qt = 2.3**((celsius-21.)/10.)
    mAlpha = (0.182 * (v- -38))/(1.-(exp(-(v- -38.)/6.)))
    mBeta  = (0.124 * (-v -38))/(1.-(exp(-(-v -38.)/6.)))
    mTau = (1./(mAlpha + mBeta)) / qt / 6.0 # NOTE: six times faster is division
    return mTau

In [ ]:
plt.figure()
plt.plot(v, tau_m, 'b', label='Kitano')
plt.plot(v, 1/(alpha_m(v)+beta_m(v)), 'g', label='Edgerton')
plt.plot(v, mtau(v), 'r', label='Moyer')
plt.plot(v, mtau_hay(v), 'm', label='Hay')
# plt.ylim(0.0, 0.1)
plt.xlabel('V (mV)')
plt.ylabel(r'$\tau_m$ (ms)')
plt.legend()